<a href="https://colab.research.google.com/github/GabrielMou21/projeto-bd-colab/blob/main/Projeto_BD_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PROJETO-BD-SQLITE.ipynb
# ---------------------------------------------------------
# Seção 1: Importar bibliotecas e conectar ao BD
import sqlite3
import pandas as pd  # Para exibir os resultados em formato de tabela

# Conectar ao banco de dados (em memória)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Ativar o suporte a chaves estrangeiras
cursor.execute("PRAGMA foreign_keys = ON;")
print("Conexão estabelecida e chaves estrangeiras ativadas com sucesso!")

# ---------------------------------------------------------
# Seção 2: Criação de tabelas

# Tabela de Clientes
cursor.execute('''
CREATE TABLE IF NOT EXISTS clientes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    email TEXT UNIQUE,
    data_cadastro DATE DEFAULT CURRENT_DATE
);
''')

# Tabela de Produtos
cursor.execute('''
CREATE TABLE IF NOT EXISTS produtos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    categoria TEXT,
    preco REAL CHECK(preco > 0),
    estoque INTEGER DEFAULT 0
);
''')

# Tabela de Pedidos
cursor.execute('''
CREATE TABLE IF NOT EXISTS pedidos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    cliente_id INTEGER,
    data_pedido TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    status TEXT DEFAULT 'Pendente',
    FOREIGN KEY (cliente_id) REFERENCES clientes(id) ON DELETE CASCADE
);
''')

# Tabela de Itens do Pedido
cursor.execute('''
CREATE TABLE IF NOT EXISTS itens_pedido (
    pedido_id INTEGER,
    produto_id INTEGER,
    quantidade INTEGER CHECK(quantidade > 0),
    preco_unitario REAL,
    PRIMARY KEY (pedido_id, produto_id),
    FOREIGN KEY (pedido_id) REFERENCES pedidos(id) ON DELETE CASCADE,
    FOREIGN KEY (produto_id) REFERENCES produtos(id)
);
''')

conn.commit()
print("Tabelas criadas com sucesso!")

# ---------------------------------------------------------
# Seção 3: Inserção de dados

# Inserir clientes
clientes = [
    ('Ana Silva', 'ana@email.com'),
    ('Carlos Oliveira', 'carlos@email.com'),
    ('Mariana Souza', 'mariana@email.com'),
    ('João Santos', 'joao@email.com')
]
cursor.executemany("INSERT INTO clientes (nome, email) VALUES (?, ?);", clientes)

# Inserir produtos
produtos = [
    ('Notebook Dell', 'Eletrônicos', 4500.00, 10),
    ('Smartphone Samsung', 'Eletrônicos', 2200.00, 15),
    ('Mesa de Escritório', 'Móveis', 800.00, 5),
    ('Cadeira Ergonômica', 'Móveis', 650.00, 8),
    ('Mouse Sem Fio', 'Acessórios', 120.00, 20)
]
cursor.executemany("INSERT INTO produtos (nome, categoria, preco, estoque) VALUES (?, ?, ?, ?);", produtos)

# Inserir pedidos
cursor.execute("INSERT INTO pedidos (cliente_id) VALUES (1);")  # Pedido da Ana
cursor.execute("INSERT INTO pedidos (cliente_id) VALUES (2);")  # Pedido do Carlos
cursor.execute("INSERT INTO pedidos (cliente_id, status) VALUES (3, 'Processando');")  # Pedido da Mariana

# Inserir itens dos pedidos
itens = [
    (1, 1, 1, 4500.00),  # Notebook Dell no pedido 1
    (1, 5, 2, 120.00),   # 2 Mouses no pedido 1
    (2, 2, 1, 2200.00),  # Smartphone no pedido 2
    (3, 3, 1, 800.00),   # Mesa no pedido 3
    (3, 4, 2, 650.00)    # 2 Cadeiras no pedido 3
]
cursor.executemany("INSERT INTO itens_pedido VALUES (?, ?, ?, ?);", itens)

conn.commit()
print("Dados inseridos com sucesso!")

# ---------------------------------------------------------
# Seção 4: Consultas e operações no banco de dados

print("\n--- Consulta 1: Lista de todos os clientes ---")
cursor.execute("SELECT * FROM clientes;")
df_clientes = pd.DataFrame(cursor.fetchall(), columns=['ID', 'Nome', 'Email', 'Data Cadastro'])
display(df_clientes)

print("\n--- Consulta 2: Produtos com estoque baixo (menos de 10 unidades) ---")
cursor.execute("SELECT nome, estoque FROM produtos WHERE estoque < 10;")
df_estoque_baixo = pd.DataFrame(cursor.fetchall(), columns=['Produto', 'Estoque'])
display(df_estoque_baixo)

print("\n--- Consulta 3: Pedidos com detalhes dos clientes ---")
cursor.execute('''
SELECT p.id, c.nome, p.data_pedido, p.status
FROM pedidos p
JOIN clientes c ON p.cliente_id = c.id;
''')
df_pedidos = pd.DataFrame(cursor.fetchall(), columns=['ID Pedido', 'Cliente', 'Data', 'Status'])
display(df_pedidos)

print("\n--- Consulta 4: Relatório completo de pedidos com itens ---")
cursor.execute('''
SELECT
    ped.id AS pedido_id,
    cli.nome AS cliente,
    prod.nome AS produto,
    it.quantidade,
    it.preco_unitario,
    (it.quantidade * it.prece_unitario) AS subtotal
FROM pedidos ped
JOIN clientes cli ON ped.cliente_id = cli.id
JOIN itens_pedido it ON ped.id = it.pedido_id
JOIN produtos prod ON it.produto_id = prod.id
ORDER BY ped.id;
''')
df_relatorio = pd.DataFrame(cursor.fetchall(), columns=['Pedido', 'Cliente', 'Produto', 'Quantidade', 'Preço Unitário', 'Subtotal'])
display(df_relatorio)

print("\n--- Consulta 5: Valor total por pedido ---")
cursor.execute('''
SELECT
    ped.id AS pedido,
    cli.nome AS cliente,
    SUM(it.quantidade * it.preco_unitario) AS total
FROM pedidos ped
JOIN clientes cli ON ped.cliente_id = cli.id
JOIN itens_pedido it ON ped.id = it.pedido_id
GROUP BY ped.id;
''')
df_totais = pd.DataFrame(cursor.fetchall(), columns=['Pedido', 'Cliente', 'Total'])
display(df_totais)

# ---------------------------------------------------------
# Seção 5: Operações de atualização e exclusão

print("\nAtualizando estoque após vendas...")
cursor.execute('''
UPDATE produtos
SET estoque = estoque - (
    SELECT COALESCE(SUM(quantidade), 0)
    FROM itens_pedido
    WHERE produto_id = produtos.id
)
''')
conn.commit()

print("\n--- Estoque atualizado ---")
cursor.execute("SELECT nome, estoque FROM produtos;")
df_estoque_atualizado = pd.DataFrame(cursor.fetchall(), columns=['Produto', 'Estoque'])
display(df_estoque_atualizado)

# ---------------------------------------------------------
# Seção 6: Encerramento
conn.close()
print("\nConexão com o banco de dados encerrada.")